## File input and preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('FAQs.csv')

In [ ]:
train_df

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


In [ ]:
len(train_df)

10

In [ ]:
questions = train_df['Question']
answers = train_df['Answer']

In [ ]:
FAQ = dict(zip(questions, answers))

In [ ]:
test_df = pd.read_csv('FAQs_test.csv')

In [ ]:
test_questions = test_df['Question']

In [ ]:
test_questions

0        What is the date of his death?
1           Did Einstein have siblings?
2                     Who was his wife?
3    What was Einstein's father's name?
4    At what institutions did he study?
Name: Question, dtype: object

## Sentence Transformers- multi-qa-mpnet-base-dot-v1

In [ ]:
pip install -U sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.5 MB 26.8 MB/s 
     |████████████████████████████████| 1.3 MB 53.1 MB/s 
     |████████████████████████████████| 163 kB 56.2 MB/s 
     |████████████████████████████████| 7.6 MB 40.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=237f31dc0baa8c0b784dc41e26eb21e3c18b30c58ec8e2e4d5d8f54cca831b80
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
print(test_questions[0])

What is the date of his death?


In [ ]:
# final_answers = []

In [ ]:
final_answers = []
for ques in test_questions:
  query_emb = model.encode(ques)
  doc_emb = model.encode(questions)
  scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
  #Combine docs & scores
  doc_score_pairs = list(zip(questions, scores))

  #Sort by decreasing score
  doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
  #Output passages & scores
  print(ques)
  print(doc_score_pairs[0][0])
  similar_ques = doc_score_pairs[0][0]
  # print(FAQ[similar_ques])

  final_answers.append(FAQ[similar_ques])
  # for doc, score in doc_score_pairs:
  #     print(score, doc)
  #     print(type(doc_score_pairs))
  # break
  

What is the date of his death?
When did he die?
Did Einstein have siblings?
Did he have any sisters and brothers?
Who was his wife?
Did he marry and have children?
What was Einstein's father's name?
When was Albert Einstein born?
At what institutions did he study?
Where did he receive his education?


In [ ]:
final_test = list(zip(test_questions, final_answers))

In [ ]:
final_test_df = pd.DataFrame(final_test, columns=['Questions','Answers'])

In [ ]:
final_test_df

,Questions,Answers
0,What is the date of his death?,"He died 18 April 1955 in Princeton, New Jersey..."
1,Did Einstein have siblings?,He had one sister named Maja.
2,Who was his wife?,He was married to Mileva Marić between 1903 an...
3,What was Einstein's father's name?,Albert Einstein was born on 14 March 1879.
4,At what institutions did he study?,He received his main education at the followin...


From our general knowledge **we can see our model found 4 out of 5 questions correctly. so the accuracy is about 80% in general.** Lets see if we can improve it further.

## Using BERT and sent2vec

In [ ]:
pip install sent2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
type(questions)

pandas.core.series.Series

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

In [ ]:
vectorizer = Vectorizer(pretrained_weights='distilbert-base-uncased')
vectorizer.run(questions.to_list())
vectors_bert = vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
vectorizer_test = Vectorizer(pretrained_weights='distilbert-base-uncased')
vectorizer_test.run(test_questions.to_list())
vectors_bert_test = vectorizer_test.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
dist_1 = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])

In [ ]:
selected_questions = []
for i in range(len(test_questions)):
  min_dist = 100000000
  index = -1
  for j in range(len(questions)):
    dist = spatial.distance.cosine(vectors_bert_test[i], vectors_bert[j])
    # print(dist)
    if min_dist > dist:
      min_dist = dist
      index = j
  selected_questions.append(questions[j])  

    

In [ ]:
selected_questions

['For what did he receive the Nobel Prize?',
 'For what did he receive the Nobel Prize?',
 'For what did he receive the Nobel Prize?',
 'For what did he receive the Nobel Prize?',
 'For what did he receive the Nobel Prize?']

In [ ]:
final_answers_bert = []
for qs in selected_questions:
  final_answers_bert.append(FAQ[qs])

In [ ]:
final_answers_bert

['Einstein was rewarded for his many contributions to theoretical physics, and especially for his discovery of the law of the photoelectric effect.',
 'Einstein was rewarded for his many contributions to theoretical physics, and especially for his discovery of the law of the photoelectric effect.',
 'Einstein was rewarded for his many contributions to theoretical physics, and especially for his discovery of the law of the photoelectric effect.',
 'Einstein was rewarded for his many contributions to theoretical physics, and especially for his discovery of the law of the photoelectric effect.',
 'Einstein was rewarded for his many contributions to theoretical physics, and especially for his discovery of the law of the photoelectric effect.']

In [ ]:
final_test_bert = list(zip(test_questions, final_answers_bert))

In [ ]:
final_test_bert_df = pd.DataFrame(final_test_bert, columns=['Questions','Answers'])

In [ ]:
final_test_bert_df

,Questions,Answers
0,What is the date of his death?,Einstein was rewarded for his many contributio...
1,Did Einstein have siblings?,Einstein was rewarded for his many contributio...
2,Who was his wife?,Einstein was rewarded for his many contributio...
3,What was Einstein's father's name?,Einstein was rewarded for his many contributio...
4,At what institutions did he study?,Einstein was rewarded for his many contributio...


**So, distilbert-base-uncased does not give a good result at all.**

## Best Outcome

Best outcome is found by using multi-qa-mpnet-base-dot-v1

In [ ]:
final_test_df

,Questions,Answers
0,What is the date of his death?,"He died 18 April 1955 in Princeton, New Jersey..."
1,Did Einstein have siblings?,He had one sister named Maja.
2,Who was his wife?,He was married to Mileva Marić between 1903 an...
3,What was Einstein's father's name?,Albert Einstein was born on 14 March 1879.
4,At what institutions did he study?,He received his main education at the followin...


In [ ]:
final_test_df.to_csv('FAQ_QA.csv', index= False)